# Elo Merchant

## Introduction
This notebook represents Sean Ng's submission to elo merchant. 

I got some feature engineering ideas from:

https://www.kaggle.com/denzo123/a-closer-look-at-date-variables


## Helper functions to manage memory

In [541]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [542]:
#To reset all variables
def remove_var(*varnames):
    """
    varnames are what you want to keep
    """
    globals_ = globals()
    to_save = {v: globals_[v] for v in globals_ if v not in varnames }
    to_save['my_reset'] = my_reset  # lets keep this function by default
    del globals_
    get_ipython().magic("reset")
    globals().update(to_save)

In [543]:
def find_variables():
    import sys

    # These are the usual ipython objects, including this one you are creating
    ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

    # Get a sorted list of the objects and their sizes
    sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

In [544]:
## Importing data

In [545]:
#Declaring imports
import csv
import os
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgd


In [546]:
EXPLORE = False

In [547]:
#Files
train_data_path = '../input/train.csv'
test_data_path = '../input/test.csv'
history_path = '../input/historical_transactions.csv'
merchant_path = '../input/merchants.csv'
new_transactions_path = '../input/new_merchant_transactions.csv'


In [548]:
train_df = reduce_mem_usage(pd.read_csv(train_data_path, parse_dates=["first_active_month"]))
test_df = reduce_mem_usage(pd.read_csv(test_data_path, parse_dates=["first_active_month"]))
n_train = train_df.shape[0]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)


In [549]:
merchants = pd.read_csv(merchant_path)

In [550]:
all_transactions = reduce_mem_usage(
    pd.read_csv(new_transactions_path, parse_dates=["purchase_date"])
    .append(
        pd.read_csv(history_path, parse_dates=["purchase_date"]),
        ignore_index=True
    )
)

Mem. usage decreased to 1867.06 Mb (43.7% reduction)


In [551]:
all_data = train_df.append(test_df, ignore_index=True)

/anaconda/envs/tensorflow/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


## Simple exploration

In [552]:
all_data.describe()

,feature_1,feature_2,feature_3,target
count,325540.00000,325540.000000,325540.000000,201917.000000
mean,3.10681,1.744038,0.565116,NaN
std,1.18728,0.750540,0.495742,0.000000
min,1.00000,1.000000,0.000000,-33.218750
25%,2.00000,1.000000,0.000000,-0.883301
50%,3.00000,2.000000,1.000000,-0.023438
75%,4.00000,2.000000,1.000000,0.765625
max,5.00000,3.000000,1.000000,17.968750


In [553]:
if EXPLORE:
    all_transactions.describe()

In [554]:
print("All data shape:" + str(all_data.shape))
print("-----------------------------------------------------")
print("Train data shape:" + str(train_df.shape))
print("Test data shape:" + str(test_df.shape))
print("=====================================================")
print("All transactions data shape:" + str(all_transactions.shape))
print("-----------------------------------------------------")
print("Old transactions data shape:" + str(history_df.shape))
print("New transactions data shape:" + str(new_transactions_df.shape))

All data shape:(325540, 6)
-----------------------------------------------------
Train data shape:(201917, 6)
Test data shape:(123623, 5)
All transactions data shape:(31075392, 14)
-----------------------------------------------------
Old transactions data shape:(29112361, 14)
New transactions data shape:(31075392, 14)


In [555]:
def get_df_name(df):
    for x in globals():
        if globals()[x] is df:
            return x

In [556]:
#Finding number of null values
def count_nulls(df_list):
    result = {}
    for df in df_list:
        vals = {}
        for key in df.keys():
            count = df[key].isnull().sum()
            if count > 0:
                vals[key] = count
        name = get_df_name(df)
        result[name] = vals
    return pd.DataFrame.from_dict(result, dtype=int)
count_nulls([all_data, all_transactions, merchants])


,all_data,all_transactions,merchants
avg_sales_lag12,NaN,NaN,13.0
avg_sales_lag3,NaN,NaN,13.0
avg_sales_lag6,NaN,NaN,13.0
category_2,NaN,2764609.0,11887.0
category_3,NaN,234081.0,NaN
first_active_month,1.0,NaN,NaN
merchant_id,NaN,164697.0,NaN
target,123623.0,NaN,NaN


# Handling Nulls

1. Viewing the distribution of non-null values

In [557]:
#putting each col into a bucket
def classify_categories(df, cols):
    discrete = []
    continuous = []
    for col in cols:
        
        length = len(df[col].unique())
        if length <= 25 :
            discrete.append(col)
        else:
            continuous.append(col)
    return discrete, continuous


In [558]:
def plot_na(df, cols):
    discrete, continuous = classify_categories(df, cols)
    all_count = len(continuous) + len(discrete)
    #Distributions of continuous dVata
    fig, axes = plt.subplots(nrows=all_count, ncols=1, figsize=(3,5*all_count))
    if all_count == 1:
        axes = [axes]
    for i, col_name in enumerate(discrete):
        df[col_name].value_counts().plot(kind='bar', ax=axes[i], title = col_name)
    for i, col_name in enumerate(continuous):
        df[col_name].plot(kind='density', ax=axes[i+len(discrete)], title = col_name)
    


In [559]:
if EXPLORE:
    plot_na(all_transactions, ["category_2", "category_3"])

In [560]:
if EXPLORE:
    plot_na(merchants, ['avg_sales_lag12','avg_sales_lag3','avg_sales_lag6','category_2'])

In [561]:
if EXPLORE:
    #Plot date to get a rough idea
    all_data.set_index(['first_active_month']).groupby('first_active_month').card_id.count().plot()

In [562]:
if EXPLORE:
    merchants.avg_sales_lag3[merchants.avg_sales_lag3!=np.nan].sort_values(ascending=False)[500:].plot()

# Feature Engineering

## Converting date to corresponding date difference

In [563]:

from datetime import datetime
from dateutil import relativedelta
def convert_date_to_timediff(df, cols):
    today = datetime.today()
    for col in cols:
        diff = df[col].dropna().map(lambda x: relativedelta.relativedelta(today,x))
        diff = diff.map(lambda x: x.years * 12 + x.months).astype('int64')
        df[col] = diff
    return df 

In [564]:
#all_data = convert_date_to_timediff(all_data, ["first_active_month"])

In [565]:
#all_transactions = convert_date_to_timediff(all_transactions, ["purchase_date"])


## Making the day of week relevant

In [566]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [567]:
%%cython
def f(x):
    return x.weekday()

In [568]:
all_transactions["day_of_week"] = all_transactions["purchase_date"].apply(f)


## Seperating data by flag because apparently they have significant difference

In [569]:
flag = all_transactions["authorized_flag"].copy()

## LabelEncode/ Hot Encode the necessary values


In [570]:
from sklearn.preprocessing import LabelEncoder
def LabelEncodeCols(df, cols):
    new_df = df.copy()
    for col in cols:
        lbl = LabelEncoder()
        new_df[col] = lbl.fit_transform(new_df[col])
    return new_df



In [571]:
all_transactions = LabelEncodeCols(all_transactions, ["authorized_flag", "category_1"])

In [572]:
all_transactions = pd.get_dummies(all_transactions, columns=["category_2", "category_3", "day_of_week"])

In [573]:
all_transactions_flagged.shape

(28558483, 27)

In [574]:
all_transactions.shape

(31075392, 27)

In [575]:
all_transactions_flagged = all_transactions[flag=='Y']

In [576]:
all_transactions = all_transactions[flag=='N']

In [577]:
#TODO: Consider if I should flag out data that is not authoized (authorized flag)
def aggregate(history):
    
    agg = {
        "authorized_flag":["mean", "var"],
        "category_1":["mean", "var"],
        "category_2_1.0":["mean"], 
        "category_2_2.0":["mean"],
        "category_2_3.0":["mean"],
        "category_2_4.0":["mean"],
        "category_2_5.0":["mean"],
        "category_3_A":["mean"],
        "category_3_B":["mean"],
        "category_3_C":["mean"],
        "city_id":['nunique'],
        "month_lag":[np.ptp, "mean"],
        "installments":["max", "mean", "var"],
        "merchant_id":['nunique'],
        "state_id":['nunique'],
        "subsector_id":["nunique"],
        "purchase_amount":["mean", "std", np.ptp],
        "day_of_week_0":["mean"],
        "day_of_week_1":["mean"],
        "day_of_week_2":["mean"],
        "day_of_week_3":["mean"],
        "day_of_week_4":["mean"],
        "day_of_week_5":["mean"],
        "day_of_week_6":["mean"],
        "card_id":["size"]
    }
    
    new_df = history.groupby("card_id").agg(agg)
    #Replace columns
    new_df.columns = [' '.join(col).strip() for col in new_df.columns.values]
    return new_df
    

In [578]:
aggregate_transactions = aggregate(all_transactions)
aggregate_flagged_transactions = aggregate(all_transactions_flagged)


In [579]:
aggregate_flagged_transactions.columns = ["flag_" + x for x in aggregate_flagged_transactions.keys()]

In [580]:
all_data = pd.merge(all_data, aggregate_transactions, on='card_id', how='left')

In [581]:
all_data = pd.merge(all_data, aggregate_flagged_transactions, on='card_id', how='left')


In [582]:
#Consider adding other columns. i.e. grouping the values across time (!!!!!)

In [583]:
#tuning hyperparameters

In [584]:
#Features that others added
#Handling month_lag?

1. Month difference between today and purchase date (might be more significant than just date(?). I need to be able to convert to something significant anyway
2. Aggregation based statistics


In [585]:
months = all_data['first_active_month']
del all_data['first_active_month']

In [586]:
all_data.fillna(0, inplace=True)

# Training Phase

In [587]:
all_data = reduce_mem_usage(all_data)

Mem. usage decreased to 46.88 Mb (69.6% reduction)


In [588]:
if "target" in all_data.keys():
    target = all_data["target"][:n_train]
if "card_id" in all_data.keys():
    card_ids = all_data["card_id"]
to_be_deleted = ["target", "card_id"]
for col in to_be_deleted:
    if col in all_data.keys():
        del all_data[col]
train = all_data[:n_train]
test = all_data[n_train:]

In [ ]:
# Selecting the right features
param = {'num_leaves': 111,
         'min_data_in_leaf': 149, 
         'objective':'regression',
         'max_depth': 9,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.7522,
         "bagging_freq": 1,
         "bagging_fraction": 0.7083 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.2634,
         "random_state": 133,
         "verbosity": -1}

import time
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
def run_lgbm_cv():
    categorical_feats = ["feature_2", "feature_3"]
    folds = KFold(n_splits = 5, shuffle=True, random_state=15)
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    start = time.time()
    feature_importance_df = pd.DataFrame()
    features = all_data.keys()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=categorical_feats
                              )
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=categorical_feats
                              )

        num_round = 10000
        clf = lgb.train(param,
                        trn_data,
                        num_round,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=100,
                        early_stopping_rounds = 200)

        oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

    print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))
    return feature_importance_df, predictions

In [ ]:
if EXPLORE:
    plt.figure(figsize=(14,25))
    plt.title('LightGBM feature importances')
    sns.barplot(x="importance", y="feature", data=feature_importances)

In [ ]:
sub_df = pd.DataFrame({"card_id":test_df.card_id.values})
sub_df["target"] = predictions
sub_df.to_csv("submit.csv", index=False)

In [ ]:
if EXPLORE:
    corrmat = all_data.corr()
    plt.subplots(figsize=(12,9))
    sns.heatmap(corrmat, vmax=0.9, square=True)

In [505]:
n_folds=5
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, target, scoring="neg_mean_squared_error", cv = kf, verbose=1))
    return(rmse)

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb


In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.001, random_state=1, tol=0.3))

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.5, l1_ratio=.9, random_state=3))
ridge = make_pipeline(RobustScaler(), Ridge(alpha=10))

In [ ]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ridge)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

#Doesn't seem to work
KRR =  KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [530]:
GBoost = GradientBoostingRegressor(n_estimators=100, learning_rate=0.05,
                                   max_depth=3, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Gradient Boosting score: 3.8617 (0.0358)



[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.2min finished


In [531]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             n_estimators=100,
                             random_state =7, nthread = -1)

score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: 3.8381 (0.0339)



[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.3min finished


In [63]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

[CV]  ................................................................
[CV] ...................... , score=-14.596460013018843, total=   3.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.5s remaining:    0.0s


[CV] ...................... , score=-14.641113285954846, total=   3.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.7s remaining:    0.0s


[CV] ...................... , score=-14.518480013374495, total=   3.2s
[CV]  ................................................................
[CV] ...................... , score=-14.651496713617208, total=   3.3s
[CV]  ................................................................
[CV] ...................... , score=-15.217078129992197, total=   3.2s
LGBM score: 3.8372 (0.0325)



[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   16.5s finished


{'max_depth': 3, 'n_estimators': 100}